In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import torch
import string
import sys as s
from collections import defaultdict


def p(thing):
    s.stdout.write(thing)


def isnan(x):
    if type(x) == str:
        return False
    try:
        int(x)
        return False
    except:
        return True


class pbar:
    def __init__(self, length, total, frac=False):
        self.length = length
        self.total = total
        if not frac:
            p('|' + '.' * length + '|')
        else:
            pass
        self.count = 0
        self.thresh = total // length
        self.all_count = 0
        self.n = 0

    def frac(self):
        if self.n == self.total - 1:
            p('\n')
            return
        if self.n > 0:
            p('\b' * self.len)
        string = f'{self.n}/{self.total}'
        p(string)
        self.len = len(string)
        self.n += 1


class fbar:
    def __init__(self, length):
        self.length = length - 1
        self.count = 0
        self.slen = 0

    def step(self):
        if self.count == self.length:
            p('\b' * self.slen)
            print(f'{self.count}/{self.length}')
            return
        p('\b' * self.slen)
        s = f'{self.count}/{self.length} '
        self.slen = len(s)
        p(s)
        self.count += 1


newsdf = pd.read_csv('./corona_fake.csv')
titles = newsdf['title']
text = newsdf['text']
labels = newsdf['label']
letters = string.ascii_letters + ' !0123456789?'


def enum1(x):
    for i in range(len(x)):
        yield (i + 1, x[i])


vocab = {letter: i for i, letter in enum1(letters)}
vocab = defaultdict(lambda: 0, vocab)
#print(vocab[''],vocab['7'])


def word2tensor(word):
    tens = [vocab[i] for i in word]
    if len(tens)>0:
      return torch.tensor(tens)
    else:
      return torch.tensor([0])


def stack(data):
    result = []
    for i in range(len(data)):
        try:
            result.append(word2tensor(data[i]))
        except:
            result.append(torch.zeros(1))
    return result


tensortext = stack(text)
tensorlabel = []
for label in labels:
    if label == 'Fake' or label == 'fake':
        tensorlabel.append(torch.zeros(1))
    elif label == 'TRUE':
        tensorlabel.append(torch.ones(1))
    else:
        # print(label,'n')
        tensorlabel.append(-1)
tensortitles = stack(titles)


def greatest_len(x):
    great = len(x[0])
    for i in x:
        if len(i) > great:
            great = i
        return great


def pad_to(x, length):
    return torch.nn.functional.pad(x, (0, length - len(x)))


def clean(thing, length):
    new = []
    for sentence in thing:
        new.append(pad_to(sentence, length))
    return new


titlemax = 150
textmax = 400
ptext = clean(tensortext, textmax)
ptitle = clean(tensortitles, titlemax)
len(ptitle), len(ptext), len(tensorlabel)
all_data = []


def batcher(x, y, batch):
    c = batch
    re = []
    for i in range(0, len(x), batch):
        re.append((x[i:c], y[i:c]))
    return re


for j in range(len(ptitle)):
    if tensorlabel[j] < 0:
        # print(tensorlabel[j])
        pass
    elif sum(ptext).sum() == 0 and sum(ptitle).sum() == 0:
        pass
    else:
        all_data.append((ptitle[j], ptext[j], tensorlabel[j]))
train_data = all_data[264:]
val_data = all_data[:264]

def list2tensor(l):
  out=torch.empty(len(l),*l[0].shape)
  return torch.cat(l,out=out)
# print('done')
import math
split = 100
val_titles = list(titles)[:split]
val_text = list(text)[:split]
train_titles = list(titles)[split:]
train_labels = tensorlabel[split:]
train_text = list(text)[split:]


In [ ]:

def isnan(x):
  if type(x)==str:
    return False
  try:
    int(x)
    return False
  except:
    return True

def getwords(b):
  r=[]
  for i in b:
    if not isnan(i):
      for j in i.split(' '):
        r.append(j.replace('\n','').replace('"','').replace("'",''))
  return r
dim=100
all=getwords(titles)+getwords(text)
dumb=nn.Embedding(len(vocab)+1,dim)
def embed(input):
  with torch.no_grad():
    return torch.sum(dumb(word2tensor(input).long()), dim=0)
def contains(x,container):
  for i in container:
    if (i==x).sum()==i.numel():
      return True
  return False
def remove(x):
  done=[]
  for i in x:
    if not contains(i,done):
      done.append(i)
    del i
  return done
print(all[0])
embed(all[0])
#embedded_all=[embed(i) for i in all]

#len(torch.stack(embedded_all).unique())-len(set(all))
#len(remove(embedded_all)),len(set(all))


Due


tensor([-4.9217, -2.2205, -0.8631, -1.0614,  1.7095, -2.1590,  1.8231,  2.5044,
        -0.8691,  1.0112,  5.4802, -1.2348, -2.1486, -1.2807,  1.5651, -0.2682,
        -1.7874,  0.2331, -0.5638,  1.7966, -0.6699,  1.3899, -0.8097, -1.3967,
        -0.2658,  0.9888,  4.0989,  0.4038,  0.2429,  2.0095, -1.4716, -4.7322,
         2.0807, -0.1699,  1.7101, -1.6154, -0.7053,  0.1353,  0.3155, -1.9471,
         0.5579, -0.8717, -2.1674, -0.2329,  1.5079,  0.4715, -2.1382,  0.2069,
        -0.0545,  3.0003, -0.9658, -1.2989,  1.9854,  2.7862, -2.0955, -0.2863,
        -1.8019, -2.7903, -3.1690,  4.3904,  0.9915,  3.5133, -1.4320,  2.8839,
         1.1204,  1.3776,  0.0843,  0.3554, -0.3815, -0.1165, -1.2694,  1.8540,
        -1.3137,  0.7076,  0.1996, -0.5168, -0.1836, -2.5682,  0.5509, -0.5149,
        -0.3655,  3.2183, -1.1762, -0.1546,  0.9054,  1.5223, -0.2034,  0.3262,
         0.2552,  0.2474,  0.6960, -0.9768, -2.4917, -1.4212,  1.9790,  0.8063,
         1.5293,  1.0441,  0.6689,  0.85

In [ ]:
j=torch.zeros(2,3)
k=torch.ones(2,3)
a=torch.stack([j,j,k,k,torch.randn(2,3)])
a.unique()

tensor([-1.5019, -0.5644, -0.3091, -0.2580,  0.0000,  0.6600,  0.9215,  1.0000])

In [ ]:
class Bags(nn.Module):
  def __init__(self,embed=20,mid=100,out=1):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed,sparse=True)
    self.fc=nn.Linear(embed,out)
    self.fc2=nn.Linear(mid,out)
  def forward(self,x,y):
    numbers=self.embed(torch.cat((x,y)).reshape(1,-1).long(),None)
    #print(numbers)
    return torch.sigmoid(self.fc(numbers).reshape(-1))
    #o1=nn.functional.relu(self.fc(numbers))
    #return torch.sigmoid(self.fc2(o1).reshape(-1))
def traindumb():
  model=Bags()
  loss_func=nn.BCELoss()
  o=torch.optim.SGD(model.parameters(),0.00003)
  for epoch in range(400):
    bar=fbar(len(test_data))
    for title,text,label in test_data:
      pred=model(title,text)
      loss=loss_func(pred,label)
      model.zero_grad()
      o.zero_grad()
      loss.backward()
      o.step()
      #bar.step()
    if epoch%10==0:
      count=0
      model.eval()
      with torch.no_grad():
        for i,j,k in val_data:
          if torch.round(model(i,j))==k:
            count+=1
      print(count/len(val_data))
      print(loss)
      model.train()
traindumb()

0.3371212121212121
tensor(0.5821, grad_fn=<BinaryCrossEntropyBackward>)
0.3371212121212121
tensor(0.5852, grad_fn=<BinaryCrossEntropyBackward>)
0.3371212121212121
tensor(0.5875, grad_fn=<BinaryCrossEntropyBackward>)
0.3371212121212121
tensor(0.5892, grad_fn=<BinaryCrossEntropyBackward>)
0.3409090909090909
tensor(0.5903, grad_fn=<BinaryCrossEntropyBackward>)
0.36363636363636365
tensor(0.5910, grad_fn=<BinaryCrossEntropyBackward>)
0.3712121212121212
tensor(0.5913, grad_fn=<BinaryCrossEntropyBackward>)
0.38257575757575757
tensor(0.5913, grad_fn=<BinaryCrossEntropyBackward>)
0.39015151515151514
tensor(0.5910, grad_fn=<BinaryCrossEntropyBackward>)
0.4090909090909091
tensor(0.5906, grad_fn=<BinaryCrossEntropyBackward>)
0.4128787878787879
tensor(0.5899, grad_fn=<BinaryCrossEntropyBackward>)
0.42045454545454547
tensor(0.5891, grad_fn=<BinaryCrossEntropyBackward>)
0.4166666666666667
tensor(0.5883, grad_fn=<BinaryCrossEntropyBackward>)
0.42424242424242425
tensor(0.5873, grad_fn=<BinaryCrossEntro

KeyboardInterrupt: ignored

In [ ]:
class meh(nn.Moudle):
  def __init__(self,embed_dim=20,mid=100,out=1):
    super().__init__()
    self.embed=nn.Embedding(len(vocab)+1,embed_dim)
    self.encoderf=nn.GRUCell(embed,mid)
    self.encoderb=nn.GRUCell(embed_dim,mid)
    self.decoder=nn.LSTM(mid,mid)
    self.v=torch.randn(10,10,requires_grad=True)
    self.wa=torch.randn(10,1,requires_grad=True)
  def forward(self,x,y):
    words = [torch.mean(self.embed(word2tensor(i).long()), dim=0) for i in input.split(' ')]
    h1=torch.zeros(1,1,100)
    h2=torch.zeros(1,1,100)
    hiddens=[]
    for i in range(len(words)):
      h1=self.encoderf(words[i],h1)
      h2=self.encoderb(words[len(words)-1-i],h2)
      h=torch.cat((h1,h2))
      hiddens.append(h)
      u=

    hiddens=torch.stack(hiddens)
    a=


In [ ]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable as v
class Pos_encode(nn.Module):
  def __init__(self, vocab_size, d,max_len):
      super().__init__()
      pe=torch.zeros(max_len,d)
      for position in range(max_len):
        for i in range(0,d,2):
          pe[pos,i]=math.sin(pos/(10000**(2*i/d)))
          pe[pos,i+1]=math.sin(pos/(10000**(2*i/d)))
      pe=pe.unsqueeze(0) # is now shape (1,max_len,d)
      self.register_buffer('pe',pe) #ties pe to the class but not to the training
      self.d=d
  def forward(self, x):
    x=x*math.sqrt(self.d)
    seq_len=x.size(1)
    x=x+v(self.pe[:,:seq_len],requires_grad=False) #takes everything 
    return x

In [ ]:
a=torch.randn(8,5)
print(a)
print('\n')
a[:,:8]

tensor([[-0.8973, -0.5652, -0.8233, -0.0614, -0.8873],
        [-0.4537, -1.1318, -1.5534, -1.8104, -0.1857],
        [-0.9593,  1.3768, -0.0455,  0.3655,  0.6119],
        [ 0.6518,  0.6511,  0.1369, -1.0735,  0.5867],
        [-1.4745, -0.0503, -0.4745,  0.6663, -0.6059],
        [ 0.1933,  0.3581,  0.6043, -1.1222,  0.9225],
        [ 0.1007,  2.0358, -0.1945,  0.1555, -0.1500],
        [-0.8011, -0.0282, -0.1909,  0.8176,  0.7197]])




tensor([[-0.8973, -0.5652, -0.8233, -0.0614, -0.8873],
        [-0.4537, -1.1318, -1.5534, -1.8104, -0.1857],
        [-0.9593,  1.3768, -0.0455,  0.3655,  0.6119],
        [ 0.6518,  0.6511,  0.1369, -1.0735,  0.5867],
        [-1.4745, -0.0503, -0.4745,  0.6663, -0.6059],
        [ 0.1933,  0.3581,  0.6043, -1.1222,  0.9225],
        [ 0.1007,  2.0358, -0.1945,  0.1555, -0.1500],
        [-0.8011, -0.0282, -0.1909,  0.8176,  0.7197]])

In [ ]:
class ahfu(nn.Module):
  def __init__(self,embed=20,mid=100,out=1,qkv=10):
    super().__init__()
    self.fc=nn.Linear(mid,out)
    self.embed=nn.Embedding(len(vocab)+1,embed)
    self.wq=nn.Linear(embed,qkv,bias=False)
    self.wk=nn.Linear(embed,qkv,bias=False)
    self.wv=nn.Linear(embed,qkv,bias=False)
    self.decoder=nn.GRUCell(qkv,mid)
    self.embed_dim=embed
    self.qkv=qkv
    self.mid=mid
  def forward(self,input):
    words = [torch.mean(self.embed(word2tensor(i).long()), dim=0) for i in input.split(' ')]
    z=[]
    #[print(i,word2tensor(i)) for i in input.split(' ')]
    for i in words:
      i=i.reshape(1,-1)
      q=self.wq(i)
      k=self.wk(i)
      v=self.wv(i)
      score=torch.mm(q,k.t())
      score=score/math.sqrt(self.embed_dim*self.qkv)
      nscore=torch.nn.functional.softmax(score,dim=-1)
      z.append(nscore*v)
    h=torch.zeros(1,self.mid)
    for j in z:
      h=self.decoder(j,h)
    return torch.sigmoid(self.fc(h))
def traintitl(data, vals, tlabel, vald=None):
    if vald is None:
        vald = data
    model = ahfu()
    lof = torch.nn.BCELoss()
    o = torch.optim.Adam(model.parameters(), 0.0003)
    for k in range(25):
        b = fbar(len(data))
        for i in range(len(data)):
            b.step()
            if not isnan(data[i]) and int(tlabel[i]) > -1:
                pred = model(data[i])
                L1_reg = torch.tensor(0., requires_grad=True)
                for name, param in model.named_parameters():
                    if 'weight' in name:
                        L1_reg = L1_reg + torch.norm(param, 1)
                loss = lof(pred.reshape(-1), tlabel[i])
                loss=loss+1e-4*L1_reg
                model.zero_grad()
                o.zero_grad()
                loss.backward()
                o.step()
        # validation
        model.eval()
        count = 0
        bad = 0
        with torch.no_grad():
            for i in range(vals):
                pick = random.randint(0, len(vald) - 1)
                if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                    pre = model(vald[pick])
                    #print(vald[pick], torch.round(pre), int(tensorlabel[pick]))
                    if torch.round(pre) == tensorlabel[pick]:
                        count += 1
                else:
                    bad += 1
        model.train()
        valid_points = vals - bad
        print(f'Validation accuracy: {count / valid_points}')
        print(loss, f' epoch: {k}')
traintitl(train_text, len(val_text), train_labels, val_text)

0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063 

KeyboardInterrupt: ignored

In [ ]:
import math
split = 100
val_titles = list(titles)[:split]
val_text = list(text)[:split]
train_titles = list(titles)[split:]
train_labels = tensorlabel[split:]
train_text = list(text)[split:]


In [ ]:
s='"dshfhdb\n'
s.replace('"','').replace('\n','')

'dshfhdb'

In [ ]:
import requests
r=requests.get('https://github.com/cuilimeng/CoAID/blob/master/07-01-2020/NewsFakeCOVID-19_tweets.csv')
print(r.content)

b'\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-lC+Z9kBc6E9r9umj6DgEEoQP7CX8RgGJGegRUJbthY1Bus2eemD1Kkc1Wbacj7hxeuvVCuyeqfNsKZWxqt1uIw==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-942f99f6405ce84f6bf6e9a3e8380412.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-jbMIzLHRNoZiW7drVC2t2DEM6Cw2GhLM7Q/MLAKYj25kjX1oQN5cz2WxxWU

In [ ]:
from bs4 import  BeautifulSoup as b

In [ ]:
s=b(r.content)
t=s.findAll('td')

In [ ]:
def get_ids(link):
  r=requests.get(f'https://github.com/cuilimeng/CoAID/blob/master/07-01-2020/{link}')
  soup=b(r.content)
  total=soup.findAll('td')
  ids=[]
  for i in range(0,len(list(t)[2:]),3):
    ids.append(str(t[i])[4:-5])
  return ids

In [ ]:
fake_ids=[]
for i in range(0,len(list(t)[2:]),3):
  ids.append(str(t[i])[4:-5])

In [ ]:
import math
split = 100
val_titles = list(titles)[:split]
val_text = list(text)[:split]
train_titles = list(titles)[split:]
train_labels = tensorlabel[split:]
train_text = list(text)[split:]
import random

In [ ]:
import sys as s
class Better(nn.Module):
    def __init__(self, embed_dim=20, mid=100, out=1):
        super().__init__()
        self.embed = nn.Embedding(len(vocab) + 1, embed_dim,sparse=True)
        self.gru = nn.GRUCell(embed_dim, mid)
        self.fc = nn.Linear(mid, out)
        self.embed_dim = embed_dim

    def forward(self, input):
        words = [torch.mean(self.embed(word2tensor(i).long()), dim=0) for i in input.split(' ')]
        h = torch.zeros(1, 100)
        for i in words:
            if not isnan(i[0]):
                h = self.gru(i.reshape(1, -1), h)
            else:
                h = self.gru(torch.zeros(1, self.embed_dim), h)
        return torch.sigmoid(self.fc(h).reshape(-1))
def traintitle(data, vals, tlabel, vald=None):
    if vald is None:
        vald = data
    model = Better()
    lof = torch.nn.BCELoss()
    o = torch.optim.SGD(model.parameters(), 0.03)
    for k in range(25):
        b = fbar(len(data))
        for i in range(len(data)):
            b.step()
            if not isnan(data[i]) and int(tlabel[i]) > -1:
                pred = model(data[i])
                L1_reg = torch.tensor(0., requires_grad=True)
                for name, param in model.named_parameters():
                    if 'weight' in name:
                        L1_reg = L1_reg + torch.norm(param, 1)
                loss = lof(pred.reshape(-1), tlabel[i])
                #loss=loss+1e-10*L1_reg
                model.zero_grad()
                o.zero_grad()
                loss.backward()
                o.step()
        # validation
        model.eval()
        count = 0
        bad = 0
        with torch.no_grad():
            for i in range(vals):
                pick = random.randint(0, len(vald) - 1)
                if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                    pre = model(vald[pick])
                    #print(vald[pick], torch.round(pre), int(tensorlabel[pick]))
                    if torch.round(pre) == tensorlabel[pick]:
                        count += 1
                else:
                    bad += 1
        model.train()
        valid_points = vals - bad
        print(f'Validation accuracy: {count / valid_points}')
        print(loss, f' epoch: {k}')
traintitle(train_titles, len(val_text), train_labels, val_text)

0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063 

KeyboardInterrupt: ignored

In [ ]:
word_nums={j:i for i,j in enum1(all)}
word_nums=defaultdict(lambda:0,word_nums)
def tonum(sentence):
  return torch.tensor([word_nums[i] for i in sentence.split(' ')]).long()
class big(nn.Module):
  def __init__(self,embed_dim=10,mid=100,out=1):
    super().__init__()
    self.embedding=nn.Embedding(len(word_nums)+2,embed_dim,sparse=True)
    self.gru=nn.GRU(embed_dim,mid)
    self.fc=nn.Linear(mid,out)
  def forward(self,x):
    embedded=self.embedding(tonum(x))
    sequenced,h=self.gru(embedded.unsqueeze(1))
    return torch.sigmoid(self.fc(h))
train_special(big,train_text,len(val_text),train_labels,val_text,0.005)

['', '!', '!!', '!!US', '#', '#1', '#1,', '#182', '#3', '#3:Educate', '#5G', '#BigPharma,', '#BigTech', '#BillGates', '#Coronavirus,', '#Covid_19', '#Netflix,', '#OperationWarpSpeed', '#china', '#coronavirus', '#coronavirus.', '#essentialoils.', '#nCoV2019', '#prevention.', '#vaccine', '$', '$.04', '$1', '$1,000', '$1,200', '$1.09m', '$1.2', '$10', '$10.5', '$10.9', '$100', '$100.“Because', '$115', '$12', '$12,', '$125', '$128', '$12billion', '$13,000.', '$150', '$150,000', '$18', '$2', '$2,800.Controlled', '$2.4', '$20', '$20,000,', '$20.', '$200-$300', '$24,598', '$250', '$3', '$3,800,000', '$3.7', '$300', '$33.1', '$35,000', '$36', '$39,000', '$39,000,', '$4', '$4.3', '$4.5', '$4.5tn,', '$4.95', '$40', '$400.', '$45', '$450', '$454', '$483', '$5', '$50', '$50,000', '$50,000.00', '$500', '$500,000', '$55', '$56', '$575.', '$6', '$6.', '$6.5', '$650', '$675', '$7.4', '$7.6', '$700,000', '$75,', '$750', '$8', '$80', '$80.5', '$800', '$9', '$915', '%', '%,', '&', '(', '(!!!),', '(!),', 

IndexError: ignored

In [ ]:
def train_special(ob,data, vals, tlabel,vald=None,lr=0.01,optim='sgd'):
    if vald is None:
      vald = data
    model = ob()
    lof = torch.nn.BCELoss()
    if optim=='sgd':
      o = torch.optim.SGD(model.parameters(), lr)
    else:
      o=torch.optim.Adam(model.parameters(),lr)
    for k in range(25):
        b = fbar(len(data))
        for i in range(len(data)):
            b.step()
            if not isnan(data[i]) and int(tlabel[i]) > -1:
                pred = model(data[i])
                loss = lof(pred.reshape(-1), tlabel[i])
                model.zero_grad()
                o.zero_grad()
                loss.backward()
                o.step()
        # validation
        model.eval()
        count = 0
        bad = 0
        with torch.no_grad():
            for i in range(vals):
                pick = random.randint(0, len(vald) - 1)
                if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                    pre = model(vald[pick])
                    #print(vald[pick], torch.round(pre), int(tensorlabel[pick]))
                    if torch.round(pre) == tensorlabel[pick]:
                        count += 1
                else:
                    bad += 1
        model.train()
        valid_points = vals - bad
        print(f'Validation accuracy: {count / valid_points}')
        print(loss, f' epoch: {k}')
def all_train(ob):
  attention = ob()
  lossfunc = torch.nn.BCELoss()
  optim = torch.optim.Adam(attention.parameters(), 0.01)
  for wpoch in range(8):
      bar = pbar(1, len(train_data), True)
      for title, text, target in train_data:
          input = torch.cat((title, text))
          pred = attention(input)
          loss = lossfunc(pred.reshape(-1), target)
          attention.zero_grad()
          optim.zero_grad()
          loss.backward()
          optim.step()
          bar.frac()
      attention.eval()
      count = 0
      for val_title, val_text, val_target in val_data:
          with torch.no_grad():
              if torch.round(attention(torch.cat((val_title, val_text)))) == val_target:
                  count += 1
      attention.train()
      print(count / len(val_data))
      print(loss)


In [ ]:
a=torch.randn(1,93)
a.unsqueeze(-1).shape

torch.Size([1, 93, 1])

In [ ]:
import random
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.cuda.device(device)
print(device)
class convs(nn.Module):
  def __init__(self,embed_dim=30,out=1,mid=100,kernal=4):
    super().__init__()
    #self.embed=nn.Embedding(len(vocab)+1,embed_dim)
    self.conv1=nn.Conv1d(1,1,kernal)
    self.conv2=nn.Conv1d(1,1,kernal)
    self.gru=nn.GRU(1,mid)#embed_dim-2*(kernal-1)
    self.fc=nn.Linear(mid,out)
    self.mid=mid
  def forward(self,words):
    #words = [torch.mean(self.embed(word2tensor(i).long()), dim=0) for i in input.split(' ')]
    #stacked=torch.stack(words).reshape(1,-1).to(device)
    convd1=self.conv1(words.reshape(1,1,-1))
    convd2=self.conv2(convd1).reshape(1,-1)
    to_gru=convd2.t().unsqueeze(-1)
    out,h=self.gru(to_gru,torch.zeros(1,1,self.mid).to(device))
    return torch.sigmoid(self.fc(h).reshape(-1))
def train_c(ob,data, vals, tlabel, vald=None,lr=0.01):
  if vald is None:
        vald = data
  model = ob().cuda()
  lof = torch.nn.BCELoss()
  o = torch.optim.SGD(model.parameters(), lr)
  embed=nn.Embedding(len(vocab)+1,30)
  for k in range(25):
      b = fbar(len(data))
      for i in range(len(data)):
          b.step()
          if not isnan(data[i]) and int(tlabel[i]) > -1:
            tokens=torch.stack([torch.mean(embed(word2tensor(j).long()), dim=0) for j in data[i].split(' ')]).to(device)
            pred = model(tokens)
            loss = lof(pred.reshape(-1), tlabel[i].to(device))
            model.zero_grad()
            o.zero_grad()
            loss.backward()
            o.step()
      # validation
      model.eval()
      count = 0
      bad = 0
      with torch.no_grad():
          for i in range(vals):
              pick = random.randint(0, len(vald) - 1)
              if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                vtokens=torch.stack([torch.mean(embed(word2tensor(j).long()), dim=0) for j in vald[pick].split(' ')]).to(device)
                pre = model(vtokens)
                if torch.round(pre) == tensorlabel[pick].to(device):
                    count += 1
              else:
                  bad += 1
      model.train()
      valid_points = vals - bad
      print(f'Validation accuracy: {count / valid_points}')
      print(loss, f' epoch: {k}')
train_c(convs,train_text,len(val_text),train_labels,val_text,0.005)

cuda
0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device=torch.cuda.device(device)
device

In [ ]:
len(train_text[0])/4000

6.57875

In [ ]:
class M_atten(nn.Module):
  def __init__(self,heads,q,k,v,fc_dim):
    self.fc1=nn.Linear(q.size(0),fc_dim)
    self.fc2=nn.Linear(q.size(0),fc_dim)
    self.fc3=nn.Linear(q.size(0),fc_dim)
    self.final=nn.Linear(heads*)

In [ ]:
r1=torch.zeros(2,3)
r2=torch.ones(2,3)
o=torch.stack([r1,r1,r2,r2])
print(o)
o.unique()

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.]]])


tensor([0., 1.])

In [ ]:
m = nn.MaxPool1d(3)
input = torch.randn(1, 1, 8)
print(input)
print(m(input))
m(input).shape

tensor([[[ 0.3982,  0.1848,  0.0527, -0.4038, -0.0498, -0.3446, -1.8859,
          -0.8489]]])
tensor([[[ 0.3982, -0.0498]]])


torch.Size([1, 1, 2])

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class bconvs(nn.Module):
  def __init__(self,embed=30,c1=10,mid=100,out=1,kernal=3):
    super().__init__()
    self.embedding=nn.EmbeddingBag(len(vocab)+1,embed,sparse=True)
    self.conv=nn.Conv1d(1,1,kernal)
    self.gru=nn.GRU(c1,mid)
    self.fc=nn.Linear(mid,out)
    self.pool=nn.AvgPool1d(3)
    self.kernal=kernal
  def forward(self,x):
    words=torch.stack([self.embedding(word2tensor(i).reshape(1,-1)) for i in x.split(' ')]).reshape(1,1,-1).to(device)
    while words.size(-1)/self.kernal>1:
      words=self.conv(words)
    return torch.sigmoid(torch.max(words).reshape(-1))
def yt(ob,data, vals, tlabel,vald=None,lr=0.01,optim='sgd'):
if vald is None:
      vald = data
    model = ob().to(device)
    lof = torch.nn.BCELoss()
    if optim=='sgd':
      o = torch.optim.SGD(model.parameters(), lr)
    else:
      o=torch.optim.Adam(model.parameters(),lr)
    for k in range(25):
        b = fbar(len(data))
        for i in range(len(data)):
            b.step()
            if not isnan(data[i]) and int(tlabel[i]) > -1:
                pred = model(data[i])
                target=tlabel[i].to(device)
                loss = lof(pred.reshape(-1), target)
                model.zero_grad()
                o.zero_grad()
                loss.backward()
                o.step()
        # validation
        model.eval()
        count = 0
        bad = 0
        with torch.no_grad():
            for i in range(vals):
                pick = random.randint(0, len(vald) - 1)
                if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                    pre = model(vald[pick])
                    #print(vald[pick], torch.round(pre), int(tensorlabel[pick]))
                    if torch.round(pre) == tensorlabel[pick]:
                        count += 1
                else:
                    bad += 1
        model.train()
        valid_points = vals - bad
        print(f'Validation accuracy: {count / valid_points}')
        print(loss, f' epoch: {k}')

AttributeError: ignored

In [ ]:
import random

In [ ]:
import numpy as np

a=np.array(np.zeros((100,200,3)))
a.fill(256)

In [ ]:
import torch
t=torch.randn(1,1,3)
a=torch.nn.MaxPool1d(3)
a(t)

tensor([[[1.9179]]])